<a href="https://colab.research.google.com/github/AshrafRafiza/Maybank-Islamic-Gold-Account-i-MIGA-i-Scraping/blob/main/Maybank_Islamic_Gold_Account_i_(MIGA_i)_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [2]:
from urllib.parse import urlencode

import requests
from bs4 import BeautifulSoup

In [3]:
#load html code from a url
api_url = "https://www.maybank2u.com.my/maybank2u/malaysia/en/personal/rates/gold_and_silver.page"

headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36",
    "x-requested-with": "XMLHttpRequest",
}

page = requests.get(f"{api_url}", headers).text
print(page)

<html>
<head>
<META http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>Gold & Silver Counter Rates  | Maybank Malaysia</title>
<!--grid-layout-->
<!--ls:begin[stylesheet]-->
<style type="text/css">
          
          .iw_container
          {
            max-width:800px !important;
            margin-left: auto !important;
            margin-right: auto !important;
          }
          .iw_stretch
          {
            min-width: 100% !important;
          }
        </style>
<link href="/iwov-resources/grid/bootstrap.css" type="text/css" rel="stylesheet">
<!--ls:end[stylesheet]-->
<!--ls:begin[meta-keywords]-->
<meta name="keywords" content="Maybank,Maybank Malaysia,Maybank Rates,Maybank2u,Online Banking,Internet Banking,Rates,Gold,Silver,gold counter rates,silver counter rates,maybank counter rates">
<!--ls:end[meta-keywords]-->
<!--ls:begin[meta-description]-->
<meta name="description" content="Rates for Kijang Emas, Maybank Gold Investment Account and Maybank 

In [4]:
# getting Maybank Islamic Gold Account-i (MIGA-i)
soup = bs(page)
values = soup.body.findAll('td')
#print(values[15:])

goldValues = []

for item in values[15:]:
  item = item.text
  goldValues.append(item)

sellingValue = goldValues[1]
buyingValue = goldValues[2]
  
print(goldValues)
print(sellingValue)
print(buyingValue)

['For 100 grams and above', '248.26', '244.18', 'For below 100 grams ', '250.01', '243.68']
248.26
244.18


In [5]:
#get the date
desc = soup.body.findAll('p', attrs={'class': 'text-small'})
targetDesc = desc[-1].text
targetDatetime = targetDesc.replace('Effective on ','')

print(targetDesc)
print(targetDatetime)

Effective on 19 Sep 2022 10:23:45
19 Sep 2022 10:23:45


In [20]:
import pytz
import datetime as dt

kuala_lumpur=pytz.timezone('Asia/Kuala_Lumpur')
localtz=pytz.timezone('Europe/London')

now = dt.datetime.now()
#now=localtz.localize(now)
print(now)

now_in_kuala_lumpur=now.astimezone(kuala_lumpur)
print(now_in_kuala_lumpur)

2022-09-19 09:33:52.766319
2022-09-19 17:33:52.766319+08:00


In [37]:
#create a dataframe
data = pd.DataFrame({
    'DateTime': targetDatetime, 
    'TimeFetch': dt.datetime.now().astimezone(kuala_lumpur).strftime("%Y-%m-%d %H:%M:%S"), 
    'Selling': sellingValue, 
    'Buying': buyingValue, 
    'Description': targetDesc
    }, index=[0])
#data
data.dtypes

data = data.astype({'Selling':'float', 'Buying':'float'})
data['DateTime'] = pd.to_datetime(data['DateTime'])
data = data.set_index('TimeFetch')
#data.dtypes
data

,DateTime,Selling,Buying,Description
TimeFetch,,,,
2022-09-19 17:45:12,2022-09-19 10:23:45,248.26,244.18,Effective on 19 Sep 2022 10:23:45


In [ ]:
#create csv file
data.to_csv('MBB_Islamic_Gold.csv')

In [ ]:
#append rows in csv
df = pd.read_csv('MBB_Islamic_Gold.csv')

#get latest row number
#current_rows = df['No'][-1:]
#print(type(current_rows.values[0]))

# add 1  from latest row number
#append_index = current_rows.values[0] + 1
#print(append_index)

# set index in data
#data = data.drop('No'#)
#data
data.to_csv('MBB_Islamic_Gold.csv', mode='a', header=False)

In [ ]:
#saving the dataframe to a csv file
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#data.to_csv('/content/drive/My Drive/Colab Notebooks/MBBGoldData/MBB_Islamic_Gold.csv')

In [ ]:
#append rows in csv
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/MBBGoldData/MBB_Islamic_Gold.csv')

data.to_csv('/content/drive/My Drive/Colab Notebooks/MBBGoldData/MBB_Islamic_Gold.csv', mode='a', header=False)